In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

DATA COLLECTION

1. Datos de Yahoo Finance

In [30]:
sp500 = yf.Ticker("^GSPC")
gold = yf.Ticker("^GC=F")
oil= yf.Ticker("CL=F")
brent= yf.Ticker("BZ=F")

Gold no existe. El ticker no baja ninguna información. Lo sustituyo por la volatilidad del mercado de oro que se descarga de los datos de la FED de St. Louis.

Para el Sp500 utilizo los datos a partir del año 2000, ya que en años anteriores la composición por industria del SP500 era bastante diferente. A partir de ese año hay importnate participación del sector teconología. Hago lo mismo para las otras variables

In [31]:
start_date = '2000-01-01'
end_date = '2024-04-30'

sp500 = sp500.history(period="max")
sp500 =sp500.loc[start_date:end_date].copy ()
oil = oil.history(period="max")
oil = oil.loc[start_date:end_date].copy ()
brent = brent.history(period="max") 
brent = brent.loc[start_date:end_date:].copy ()


In [32]:
sp500.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-03 00:00:00-05:00,1469.250000,1478.000000,1438.359985,1455.219971,931800000,0.0,0.0
2000-01-04 00:00:00-05:00,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,0.0,0.0
2000-01-05 00:00:00-05:00,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,0.0,0.0
2000-01-06 00:00:00-05:00,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,0.0,0.0
2000-01-07 00:00:00-05:00,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,0.0,0.0


In [33]:
#Remueve las columans que no necesito
sp500 = sp500.drop(columns=['Dividends', 'Stock Splits'])
sp500.index= sp500.index.tz_localize(None) # remove timezone porque tengo otras variables que no la tienen. para uniformar

In [34]:
#Guarda el dataset en un archivo csv   
sp500.to_csv('sp500.csv', index=True)

In [35]:
#Hace lo mismo con el resto de los activos
oil = oil.drop(columns=['Dividends', 'Stock Splits', 'Open', 'High', 'Low', 'Volume'])
oil.index= oil.index.tz_localize(None) # remove timezone
oil.head()

,Close
Date,
2000-08-23,32.049999
2000-08-24,31.629999
2000-08-25,32.049999
2000-08-28,32.869999
2000-08-29,32.720001


In [36]:
# Renombr< la columna 'Close' a 'Close_oil' en el DataFrame 'oil' para que no se confunda con la del S&P 500
oil.rename(columns={'Close': 'Close_oil'}, inplace=True)

# Muestra las primeras filas para verificar el cambio
print(oil.head())


            Close_oil
Date                 
2000-08-23  32.049999
2000-08-24  31.629999
2000-08-25  32.049999
2000-08-28  32.869999
2000-08-29  32.720001


2. Datos de FRED (Banco de la Reserva Federal de St. Louis) -

Información económica diaria y que refleja expectativas del mercado. Como no existen variables fundamentales diarias relacionadas con el índice, 
incluyo variables diarias del mercado que reflejen el sentimiento de la evolución de la economía, lo cual afecta consecuentemente la evolución de las empresas y su valor.
Las variables macro y las fundamentales en general no suelen ser diarias.

In [37]:
import requests


# FRED API endpoint
url = "https://api.stlouisfed.org/fred/series/observations"

# List of series IDs to be downloaded
# CREDIT RISK AND RISK APPETITE VARIABLES # OJO --->>>> Estas variables miden el riesgo de crédito y el apetito por el riesgo de los inversores 

# DAAA = Moody's Seasoned Aaa Corporate Bond Yield
# AAA10Y = Moody's Seasoned Aaa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
# DBAA = Moody's Seasoned Baa Corporate Bond Yield
# BAA10Y = Moody's Seasoned Baa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
# BAMLH0A0HYM2 = ICE BofA US High Yield Index Option-Adjusted Spread 
# BAMLH0A2HYB = ICE BofA Single-B US High Yield Index Option-Adjusted Spread 
# BAMLH0A3HYC = ICE BofA CCC & Lower US High Yield Index Option-Adjusted Spread
# BAMLHE00EHYIOAS = ICE BofA Euro High Yield Index Option-Adjusted Spread -  

# INTEREST RATE VARIABLES
#OJO --->>>> Esta variables miden el cambio en las expectativas de inflación y los movimientos de tasas de interés

# T10Y2Y = 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity
# T10Y3M = 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity  
# T5YIE = 5-Year Breakeven Inflation Rate (Expected Inflation Rate) - 5-Year 

# EXCHANGE RATES Vs main commercial partners for the U.S.
# DEXUSEU = U.S. Dollars to Euro Spot Exchange Rate - value of one U.S. Dollar in Euros
# DEXCHUS = Chinese Yuan Renminbi to U.S. Dollar Spot Exchange Rate - value of one Chinese Yuan Renminbi in U.S. Dollars
# DEXCAUS = Canadian Dollars to U.S. Dollar Spot Exchange Rate - value of one Canadian Dollar in U.S. Dollars
# DEXMXUS= Mexican Pesos to U.S. Dollar Spot Exchange Rate - value of one Mexican Peso in U.S. Dollars

# VOLATILITY INDECES (MARKET EXPECTATION OF NEAR TERM VOLATILITY)
# VIXCLS = CBOE Volatility Index - VIX measures market expectation of near term volatility conveyed by stock index option prices.
#(GVZCLS)= CBOE Gold ETF Volatility Index 

series_ids = ['DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE','DEXUSEU','DEXCHUS','DEXCAUS','DEXMXUS','VIXCLS','GVZCLS']

# Your FRED API key
api_key = '7de04d83472c6affd4e0a1505211d3b9'

# Initialize an empty dictionary to store DataFrames
dfs = {}

# Loop through each series ID
for series_id in series_ids:
    # API parameters
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
    }
    
    # Make the API request
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response to JSON
        data = response.json()
        # Extract data for DataFrame
        dates = [item['date'] for item in data['observations']]
        values = [item['value'] for item in data['observations']]
        # Create DataFrame and convert types
        temp_df = pd.DataFrame({'Date': dates, series_id: values})
        temp_df['Date'] = pd.to_datetime(temp_df['Date'])
        temp_df[series_id] = pd.to_numeric(temp_df[series_id], errors='coerce')
        # Add the DataFrame to the dictionary
        dfs[series_id] = temp_df.set_index('Date')
    else:
        print(f"Failed to retrieve data for {series_id}: {response.status_code}")

# Combine all the series DataFrames into one DataFrame 
macroeconomic_df = pd.concat(dfs.values(), axis=1)




3. Uno los dataframes

In [38]:
# Unir macroeconomic_df con sp500 basado en sus índices de fecha
AllVar_df =sp500.join(macroeconomic_df, how='inner')

print(AllVar_df.index)
AllVar_df = AllVar_df.sort_index()

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2024-04-17', '2024-04-18', '2024-04-19', '2024-04-22',
               '2024-04-23', '2024-04-24', '2024-04-25', '2024-04-26',
               '2024-04-29', '2024-04-30'],
              dtype='datetime64[ns]', name='Date', length=6120, freq=None)


In [39]:
# Unir el DataFrame 'oil' al 'AllVar_df' que ya contiene datos combinados de 'macroeconomic_df' y 'sp500'
# Asegúrate de que 'oil' está indexado adecuadamente, si es necesario
if 'Date' in oil.columns:
    oil.set_index('Date', inplace=True)

# Especificando sufijos para resolver el conflicto de nombres de columnas
AllVar_df = AllVar_df.join(oil, how='inner', lsuffix='_AllVar', rsuffix='_oil')

# Mostrar las primeras filas para verificar las nuevas columnas
print(AllVar_df.head())

                   Open         High          Low        Close     Volume  \
Date                                                                        
2000-08-23  1498.130005  1507.199951  1489.520020  1505.969971  871000000   
2000-08-24  1505.969971  1511.160034  1501.250000  1508.310059  837100000   
2000-08-25  1508.310059  1513.469971  1505.089966  1506.449951  685600000   
2000-08-28  1506.449951  1523.949951  1506.449951  1514.089966  733600000   
2000-08-29  1514.089966  1514.810059  1505.459961  1509.839966  795600000   

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  ...  T10Y2Y  T10Y3M  \
Date                                                  ...                   
2000-08-23  7.55    1.82  8.26    2.53          6.43  ...   -0.47   -0.56   
2000-08-24  7.51    1.78  8.22    2.49          6.44  ...   -0.46   -0.56   
2000-08-25  7.52    1.79  8.23    2.50          6.42  ...   -0.46   -0.58   
2000-08-28  7.56    1.78  8.27    2.49          6.36  ...   -0.45   -0.54  

4. Calculo metricas de cambios

4.2 Calculo de las variables para análisis técnico (variables calculadas a partir de los precios de los activos). Ayudan a la toma de decisiones al dar información sobre el comportamiento de cada activo a través del tiempo.
Buscan determinar tendencias en los precios.

Medias moviles (dias h'abiles)

In [40]:
#Calculo la media 
# Periodos para las medias móviles
moving_avg_periods = [5,10, 20,200]

# Calcular y asignar cada media móvil utilizando un bucle
for period in moving_avg_periods:
    AllVar_df[f'{period}d_moving_avg'] = AllVar_df['Close'].rolling(window=period).mean()

# Mostrar las primeras filas para verificar las nuevas columnas
AllVar_df.head()



,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,DEXCHUS,DEXCAUS,DEXMXUS,VIXCLS,GVZCLS,Close_oil,5d_moving_avg,10d_moving_avg,20d_moving_avg,200d_moving_avg
Date,,,,,,,,,,,,,,,,,,,,,
2000-08-23,1498.130005,1507.199951,1489.520020,1505.969971,871000000,7.55,1.82,8.26,2.53,6.43,...,8.2793,1.4877,9.2725,17.38,NaN,32.049999,NaN,NaN,NaN,NaN
2000-08-24,1505.969971,1511.160034,1501.250000,1508.310059,837100000,7.51,1.78,8.22,2.49,6.44,...,8.2799,1.4871,9.2440,17.04,NaN,31.629999,NaN,NaN,NaN,NaN
2000-08-25,1508.310059,1513.469971,1505.089966,1506.449951,685600000,7.52,1.79,8.23,2.50,6.42,...,8.2799,1.4828,9.2320,16.53,NaN,32.049999,NaN,NaN,NaN,NaN
2000-08-28,1506.449951,1523.949951,1506.449951,1514.089966,733600000,7.56,1.78,8.27,2.49,6.36,...,8.2799,1.4845,9.2130,16.54,NaN,32.869999,NaN,NaN,NaN,NaN
2000-08-29,1514.089966,1514.810059,1505.459961,1509.839966,795600000,7.58,1.77,8.30,2.49,6.30,...,8.2799,1.4868,9.2325,16.89,NaN,32.720001,1508.931982,NaN,NaN,NaN


Media movil ultimos 10 días hábiles

In [41]:
#Calculo el promedio móvil ponderado de los ultimas 2 semanas

# Definimos los pesos decrecientes para los últimos 10 días hábiles.
weights = np.arange(1, 11)[::-1]  # Esto crea un array de numpy: [10, 9, ..., 2, 1]

# Calculamos el WMA para cada ventana de 10 días hábiles.
# Usamos apply para aplicar una función personalizada que calcula el WMA de manera vectorizada.
AllVar_df['10d_wma'] = AllVar_df['Close'].rolling(window=10).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)

# Ahora, el DataFrame 'AllVar_df' contiene una nueva columna '10d_wma' con el WMA de 10 días.
# Muestra los primeros registros para verificar.
print(AllVar_df[['Close', '10d_wma']].head(20))


                  Close      10d_wma
Date                                
2000-08-23  1505.969971          NaN
2000-08-24  1508.310059          NaN
2000-08-25  1506.449951          NaN
2000-08-28  1514.089966          NaN
2000-08-29  1509.839966          NaN
2000-08-30  1502.589966          NaN
2000-08-31  1517.680054          NaN
2000-09-01  1520.770020          NaN
2000-09-05  1507.079956          NaN
2000-09-06  1492.250000  1509.020898
2000-09-07  1502.510010  1509.418539
2000-09-08  1494.500000  1509.139617
2000-09-11  1489.260010  1508.886353
2000-09-12  1481.989990  1506.660358
2000-09-13  1484.910034  1504.753820
2000-09-14  1480.869995  1503.770554
2000-09-15  1465.810059  1499.100546
2000-09-18  1444.510010  1492.482180
2000-09-19  1459.900024  1487.495099
2000-09-20  1451.339966  1484.460556


Momentum

In [42]:
# Calcular el momentum como el cambio día a día en la columna 'Close'. Es lo mismo que el daily change pero en absoluto no porcentual
AllVar_df['Momentum'] = AllVar_df['Close'].diff()

print(AllVar_df.head())  # Mostrar las primeras filas para verificar la nueva columna

                   Open         High          Low        Close     Volume  \
Date                                                                        
2000-08-23  1498.130005  1507.199951  1489.520020  1505.969971  871000000   
2000-08-24  1505.969971  1511.160034  1501.250000  1508.310059  837100000   
2000-08-25  1508.310059  1513.469971  1505.089966  1506.449951  685600000   
2000-08-28  1506.449951  1523.949951  1506.449951  1514.089966  733600000   
2000-08-29  1514.089966  1514.810059  1505.459961  1509.839966  795600000   

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  ...  DEXMXUS  VIXCLS  \
Date                                                  ...                    
2000-08-23  7.55    1.82  8.26    2.53          6.43  ...   9.2725   17.38   
2000-08-24  7.51    1.78  8.22    2.49          6.44  ...   9.2440   17.04   
2000-08-25  7.52    1.79  8.23    2.50          6.42  ...   9.2320   16.53   
2000-08-28  7.56    1.78  8.27    2.49          6.36  ...   9.2130   1

Stochastic %K

El Stochastic %K es una medida de la posición relativa del precio de cierre actual en relación con el rango alto-bajo durante un número específico de períodos pasados. La fórmula para calcular el Stochastic %K es la siguiente:

% K = [(C-LLn)/(HHn-LLn)]x 100� 

donde:


C es el precio de cierre más reciente,

LLn es el precio más bajo de los prcios bajos (Low) en los últimos n períodos,
Hn  es el precio más alto de lso pecios altos (High) en los últimos n períodos,
n es el número de períodos utilizados para el cálculo.
El valor de n varía según el análisis, pero los valores comunes incluyen 14, 20 o cualquier otro valor dependiendo de la estrategia de trading

In [43]:
# Calcular el Stochastic %K para cada fila basado en los 20 días laborables previos
# Crear una función para encapsular el cálculo
#def calculate_stochastic_k(row_index, df):
    #if row_index < 19:  # Asegurar que haya suficientes datos para los primeros 19 registros
        #return None
    #else:
        #period = df['Close'][row_index-19:row_index+1]  # Seleccionar los últimos 20 registros hasta el índice actual
        #low_20 = period.min()
        #high_20 = period.max()
        #close_recent = df['Close'].iloc[row_index]
        #stochastic_k = ((close_recent - low_20) / (high_20 - low_20)) * 100
        #return stochastic_k

# Aplicar la función a cada fila
# Nota: Esto puede ser lento para DataFrames muy grandes debido al enfoque iterativo
#sp500['Stochastic_%K'] = [calculate_stochastic_k(i, sp500) for i in range(len(sp500))]

#print(sp500.tail())  # Mostrar las últimas filas para verificar la nueva columna
#sp500.head


# Crea una función para encapsular el cálculo del Stochastic %K
def calculate_stochastic_k(row_index, df):
    if row_index < 19:  # Asegurar que haya suficientes datos para los primeros 19 registros
        return None
    else:
        # Seleccionar los últimos 20 registros hasta el índice actual para las columnas 'High' y 'Low'
        high_period = df['High'][row_index-19:row_index+1]
        low_period = df['Low'][row_index-19:row_index+1]
        low_20 = low_period.min()
        high_20 = high_period.max()
        close_recent = df['Close'].iloc[row_index]
        
        # Calcular el Stochastic %K
        stochastic_k = ((close_recent - low_20) / (high_20 - low_20)) * 100
        return stochastic_k

# Aplica la función a cada fila
AllVar_df['Stochastic_%K'] = [calculate_stochastic_k(i, AllVar_df) for i in range(len(AllVar_df))]

# Muestra las últimas filas para verificar la nueva columna
print(AllVar_df.tail())


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-24  5084.859863  5089.479980  5047.020020  5071.629883  3656740000   
2024-04-25  5019.879883  5057.750000  4990.580078  5048.419922  3958050000   
2024-04-26  5084.649902  5114.620117  5073.140137  5099.959961  3604140000   
2024-04-29  5114.129883  5123.490234  5088.649902  5116.169922  3447450000   
2024-04-30  5103.779785  5110.830078  5035.310059  5035.689941  4082470000   

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  ...  VIXCLS  GVZCLS  \
Date                                                  ...                   
2024-04-24  5.41    0.76  6.13    1.48          3.19  ...   15.97   17.45   
2024-04-25  5.45    0.75  6.17    1.47          3.24  ...   15.37   17.32   
2024-04-26  5.39    0.72  6.12    1.45          3.16  ...   15.03   16.92   
2024-04-29  5.35    0.72  6.08    1.45          3.12  ...   14.67   

Stochastic %D

El indicador Stochastic %D es una media móvil del indicador Stochastic %K. Usualmente se calcula como la media móvil simple de los últimos 3 valores de %K. Para calcular el %D para las últimas 20 observaciones de tu DataFrame sp500, primero necesitaríamos calcular el %K como se discutió anteriormente, y luego aplicar una media móvil sobre esos valores de %K.

In [44]:

# Calcular el Stochastic %D como la media móvil simple de 3 días de %K
AllVar_df['Stochastic_%D'] = AllVar_df['Stochastic_%K'].rolling(window=3).mean()

# Si solo necesitas las últimas 20 observaciones para el %D, puedes hacer lo siguiente:
last_20_stochastic_D = AllVar_df['Stochastic_%D'][-20:]

print(last_20_stochastic_D)
AllVar_df.head(9)

Date
2024-04-03    76.922842
2024-04-04    57.726515
2024-04-05    55.575326
2024-04-08    53.829101
2024-04-09    64.992990
2024-04-10    54.961573
2024-04-11    53.283501
2024-04-12    35.318808
2024-04-15    25.095684
2024-04-16     7.141325
2024-04-17     5.118661
2024-04-18     4.821237
2024-04-19     4.569616
2024-04-22     8.741723
2024-04-23    20.099123
2024-04-24    31.278398
2024-04-25    35.328193
2024-04-26    38.522920
2024-04-29    43.766969
2024-04-30    42.643557
Name: Stochastic_%D, dtype: float64


,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,GVZCLS,Close_oil,5d_moving_avg,10d_moving_avg,20d_moving_avg,200d_moving_avg,10d_wma,Momentum,Stochastic_%K,Stochastic_%D
Date,,,,,,,,,,,,,,,,,,,,,
2000-08-23,1498.130005,1507.199951,1489.520020,1505.969971,871000000,7.55,1.82,8.26,2.53,6.43,...,NaN,32.049999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-08-24,1505.969971,1511.160034,1501.250000,1508.310059,837100000,7.51,1.78,8.22,2.49,6.44,...,NaN,31.629999,NaN,NaN,NaN,NaN,NaN,2.340088,NaN,NaN
2000-08-25,1508.310059,1513.469971,1505.089966,1506.449951,685600000,7.52,1.79,8.23,2.50,6.42,...,NaN,32.049999,NaN,NaN,NaN,NaN,NaN,-1.860107,NaN,NaN
2000-08-28,1506.449951,1523.949951,1506.449951,1514.089966,733600000,7.56,1.78,8.27,2.49,6.36,...,NaN,32.869999,NaN,NaN,NaN,NaN,NaN,7.640015,NaN,NaN
2000-08-29,1514.089966,1514.810059,1505.459961,1509.839966,795600000,7.58,1.77,8.30,2.49,6.30,...,NaN,32.720001,1508.931982,NaN,NaN,NaN,NaN,-4.250000,NaN,NaN
2000-08-30,1509.839966,1510.489990,1500.089966,1502.589966,818400000,7.58,1.77,8.30,2.49,6.28,...,NaN,33.400002,1508.255981,NaN,NaN,NaN,NaN,-7.250000,NaN,NaN
2000-08-31,1502.589966,1525.209961,1502.589966,1517.680054,1056600000,7.52,1.79,8.24,2.51,6.43,...,NaN,33.099998,1510.129980,NaN,NaN,NaN,NaN,15.090088,NaN,NaN
2000-09-01,1517.680054,1530.089966,1515.530029,1520.770020,767700000,7.51,1.83,8.22,2.54,6.48,...,NaN,33.380001,1512.993994,NaN,NaN,NaN,NaN,3.089966,NaN,NaN
2000-09-05,1520.770020,1520.770020,1504.209961,1507.079956,838500000,7.51,1.82,8.22,2.53,6.41,...,NaN,33.799999,1511.591992,NaN,NaN,NaN,NaN,-13.690063,NaN,NaN


El Índice de Fuerza Relativa

El Índice de Fuerza Relativa (RSI, por sus siglas en inglés) es un indicador de momento que mide la magnitud de los movimientos recientes de precios para evaluar condiciones de sobrecompra o sobreventa en el precio de un activo. Se calcula utilizando la fórmula:

RSI=100− 100/(1+RS)
 

donde 
�
�
RS es la relación media de días de cierre hacia arriba entre la media de días de cierre hacia abajo. Para calcularlo para los últimos 20 y 10 días, necesitamos seguir los siguientes pasos para cada ventana de tiempo:

Calcular los cambios diarios en el precio de cierre.
Separar estos cambios en ganancias (aumentos) y pérdidas (disminuciones).
Calcular el promedio de ganancias y el promedio de pérdidas sobre el período de interés (20 días y 10 días, respectivamente).
Calcular el RS, que es el promedio de ganancias dividido por el promedio de pérdidas.
Utilizar la fórmula del RSI para obtener el valor final.

In [45]:
# Separar las ganancias (positivas) de las pérdidas (negativas)
gain = AllVar_df['Momentum'].mask(AllVar_df['Momentum'] > 0, 0)
loss = -AllVar_df['Momentum'].mask(AllVar_df['Momentum'] < 0, 0)

# Calcular el RSI para los últimos 20 y 10 días
window_lengths = [20, 10]

for length in window_lengths:
    # Calcular el promedio de ganancias y pérdidas
    avg_gain = gain.rolling(window=length, min_periods=1).mean()
    avg_loss = loss.rolling(window=length, min_periods=1).mean()
    
    # Calcular RS
    rs = avg_gain / avg_loss
    
    # Calcular RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Adjuntar al DataFrame
    AllVar_df[f'RSI_{length}'] = rsi

# Mostrar las últimas 20 filas para verificar las nuevas columnas
print(AllVar_df.tail(20))



                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-03  5194.370117  5228.750000  5194.370117  5211.490234  3703250000   
2024-04-04  5244.049805  5256.589844  5146.060059  5147.209961  4075680000   
2024-04-05  5158.950195  5222.180176  5157.209961  5204.339844  3386780000   
2024-04-08  5211.370117  5219.569824  5197.350098  5202.390137  3278180000   
2024-04-09  5217.029785  5224.810059  5160.779785  5209.910156  3400680000   
2024-04-10  5167.879883  5178.430176  5138.700195  5160.640137  3845930000   
2024-04-11  5172.950195  5211.779785  5138.770020  5199.060059  3509380000   
2024-04-12  5171.509766  5175.029785  5107.939941  5123.410156  3963220000   
2024-04-15  5149.669922  5168.430176  5052.470215  5061.819824  3950210000   
2024-04-16  5064.589844  5079.839844  5039.830078  5051.410156  4006200000   
2024-04-17  5068.970215  5077.959961  5007.250000  5022.209961  

Indicador Larry Williams %R

El indicador Larry Williams %R es una medida de momentum que se utiliza para identificar niveles de sobrecompra y sobreventa en un mercado. Se calcula como un porcentaje que refleja la posición del precio de cierre relativo a su rango más alto y más bajo durante un número específico de períodos pasados. La fórmula del Larry Williams %R es:

%R= [(Hn-C)/(Hn-Ln)]x100

donde:

C es el precio de cierre actual,
Hn  es el precio más alto en los últimos n períodos,
Ln es el precio más bajo en los últimos 
�
n períodos,
�
n es el número de períodos utilizados para el cálculo.
Generalmente, se utiliza un período de 14 días para el cálculo de este indicador, pero se puede ajustar según las necesidades del análisis. Para agregar el Larry Williams %R al DataFrame sp500 para los últimos 14 días, puedes usar el siguiente código:

In [46]:
# Asumiendo que sp500 es tu DataFrame y tiene columnas 'High', 'Low', y 'Close' con los precios más altos, más bajos y de cierre

# Configurar el número de períodos para el cálculo de Williams %R
n = 10

# Calcular el precio más alto y el más bajo de los últimos n días
high_n = AllVar_df['High'].rolling(window=n).max()
low_n = AllVar_df['Low'].rolling(window=n).min()

# Calcular el Williams %R
AllVar_df['Williams_%R'] = ((high_n - AllVar_df['Close']) / (high_n - low_n)) * -100

# Mostrar las últimas filas para verificar la nueva columna
print(AllVar_df.tail())


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-24  5084.859863  5089.479980  5047.020020  5071.629883  3656740000   
2024-04-25  5019.879883  5057.750000  4990.580078  5048.419922  3958050000   
2024-04-26  5084.649902  5114.620117  5073.140137  5099.959961  3604140000   
2024-04-29  5114.129883  5123.490234  5088.649902  5116.169922  3447450000   
2024-04-30  5103.779785  5110.830078  5035.310059  5035.689941  4082470000   

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  ...  10d_moving_avg  \
Date                                                  ...                   
2024-04-24  5.41    0.76  6.13    1.48          3.19  ...     5058.904004   
2024-04-25  5.45    0.75  6.17    1.47          3.24  ...     5043.839990   
2024-04-26  5.39    0.72  6.12    1.45          3.16  ...     5041.494971   
2024-04-29  5.35    0.72  6.08    1.45          3.12  ...     5046.9

Moving Average Convergence/Divergence Oscillator

The MACD turns two trend-following indicators, moving averages, into a momentum oscillator by subtracting the longer moving average from the shorter one. As a result, the MACD offers the best of both worlds: trend following and momentum. The MACD fluctuates above and below the zero line as the moving averages converge, cross and diverge. Traders can look for signal line crossovers, centerline crossovers and divergences to generate signals. Because the MACD is unbounded, it is not particularly useful for identifying overbought and oversold levels.

In [47]:
#https://school.stockcharts.com/doku.php?id=technical_indicators:moving_average_convergence_divergence_macd


# Calcular las EMAs de 12 y 26 períodos para la columna 'Close'
ema12 = AllVar_df['Close'].ewm(span=12, adjust=False).mean()
ema26 = AllVar_df['Close'].ewm(span=26, adjust=False).mean()

# Calcular el MACD restando la EMA de 26 períodos de la EMA de 12 períodos
AllVar_df['MACD'] = ema12 - ema26

# Calcular la línea de señal como la EMA de 9 períodos del MACD
AllVar_df['Signal_Line'] = AllVar_df['MACD'].ewm(span=9, adjust=False).mean()

# Mostrar las últimas filas para verificar las nuevas columnas
print(AllVar_df.tail())
AllVar_df.head()


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-24  5084.859863  5089.479980  5047.020020  5071.629883  3656740000   
2024-04-25  5019.879883  5057.750000  4990.580078  5048.419922  3958050000   
2024-04-26  5084.649902  5114.620117  5073.140137  5099.959961  3604140000   
2024-04-29  5114.129883  5123.490234  5088.649902  5116.169922  3447450000   
2024-04-30  5103.779785  5110.830078  5035.310059  5035.689941  4082470000   

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  ...  200d_moving_avg  \
Date                                                  ...                    
2024-04-24  5.41    0.76  6.13    1.48          3.19  ...      4683.744802   
2024-04-25  5.45    0.75  6.17    1.47          3.24  ...      4686.790603   
2024-04-26  5.39    0.72  6.12    1.45          3.16  ...      4689.929602   
2024-04-29  5.35    0.72  6.08    1.45          3.12  ...      

,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,200d_moving_avg,10d_wma,Momentum,Stochastic_%K,Stochastic_%D,RSI_20,RSI_10,Williams_%R,MACD,Signal_Line
Date,,,,,,,,,,,,,,,,,,,,,
2000-08-23,1498.130005,1507.199951,1489.520020,1505.969971,871000000,7.55,1.82,8.26,2.53,6.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2000-08-24,1505.969971,1511.160034,1501.250000,1508.310059,837100000,7.51,1.78,8.22,2.49,6.44,...,NaN,NaN,2.340088,NaN,NaN,0.000000,0.000000,NaN,0.186674,0.037335
2000-08-25,1508.310059,1513.469971,1505.089966,1506.449951,685600000,7.52,1.79,8.23,2.50,6.42,...,NaN,NaN,-1.860107,NaN,NaN,44.286213,44.286213,NaN,0.182416,0.066351
2000-08-28,1506.449951,1523.949951,1506.449951,1514.089966,733600000,7.56,1.78,8.27,2.49,6.36,...,NaN,NaN,7.640015,NaN,NaN,15.710088,15.710088,NaN,0.786461,0.210373
2000-08-29,1514.089966,1514.810059,1505.459961,1509.839966,795600000,7.58,1.77,8.30,2.49,6.30,...,NaN,NaN,-4.250000,NaN,NaN,37.974069,37.974069,NaN,0.911722,0.350643


4.3 Cambios diarios
Trabajo con daily changes y no números absolutos porque ARIMA y modelos en general trabajan mejor con series estacionarias. Para observaciones diarias, los cambios diarios son una forma común de estacionarizar la serie temporal.

In [48]:
# Calcular los retornos diarios 
retornos_diarios = AllVar_df[['Close']].pct_change()

# Multiplicar por 100 para obtener porcentajes
retornos_diarios = retornos_diarios * 100

# Añadir la columna de retornos diarios al DataFrame original
# Nombrando la columna como 'Retornos Diarios'
AllVar_df['Daily Returns'] = retornos_diarios




In [49]:
columns_to_change = [
    'DAAA', 'AAA10Y', 'DBAA', 'BAA10Y', 'BAMLH0A0HYM2', 'BAMLH0A2HYB', 
    'BAMLH0A3HYC', 'BAMLHE00EHYIOAS', 'T10Y2Y', 'T10Y3M', 'T5YIE', 'DEXUSEU', 
    'DEXCHUS', 'DEXCAUS', 'DEXMXUS', 'VIXCLS', 'GVZCLS'
]
for column in columns_to_change:
    new_column_name = f'{column}_daily_change'
    AllVar_df[new_column_name] = AllVar_df[column].pct_change() * 100

print(AllVar_df.head())


                   Open         High          Low        Close     Volume  \
Date                                                                        
2000-08-23  1498.130005  1507.199951  1489.520020  1505.969971  871000000   
2000-08-24  1505.969971  1511.160034  1501.250000  1508.310059  837100000   
2000-08-25  1508.310059  1513.469971  1505.089966  1506.449951  685600000   
2000-08-28  1506.449951  1523.949951  1506.449951  1514.089966  733600000   
2000-08-29  1514.089966  1514.810059  1505.459961  1509.839966  795600000   

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  ...  \
Date                                                  ...   
2000-08-23  7.55    1.82  8.26    2.53          6.43  ...   
2000-08-24  7.51    1.78  8.22    2.49          6.44  ...   
2000-08-25  7.52    1.79  8.23    2.50          6.42  ...   
2000-08-28  7.56    1.78  8.27    2.49          6.36  ...   
2000-08-29  7.58    1.77  8.30    2.49          6.30  ...   

            BAMLHE00EHYIOAS_dail

C:\Users\lgarn\AppData\Local\Temp\ipykernel_6036\2385237343.py:8: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  AllVar_df[new_column_name] = AllVar_df[column].pct_change() * 100
C:\Users\lgarn\AppData\Local\Temp\ipykernel_6036\2385237343.py:8: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  AllVar_df[new_column_name] = AllVar_df[column].pct_change() * 100
C:\Users\lgarn\AppData\Local\Temp\ipykernel_6036\2385237343.py:8: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling p

In [50]:
# Calcula los cambios diarios en el precio del oil.
AllVar_df['oil_daily_changes'] = AllVar_df['Close_oil'].pct_change() * 100  # Multiplicar por 100 para convertir a porcentaje


In [51]:
AllVar_df.head()

,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,T10Y2Y_daily_change,T10Y3M_daily_change,T5YIE_daily_change,DEXUSEU_daily_change,DEXCHUS_daily_change,DEXCAUS_daily_change,DEXMXUS_daily_change,VIXCLS_daily_change,GVZCLS_daily_change,oil_daily_changes
Date,,,,,,,,,,,,,,,,,,,,,
2000-08-23,1498.130005,1507.199951,1489.520020,1505.969971,871000000,7.55,1.82,8.26,2.53,6.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-08-24,1505.969971,1511.160034,1501.250000,1508.310059,837100000,7.51,1.78,8.22,2.49,6.44,...,-2.127660,0.000000,NaN,0.680272,0.007247,-0.040331,-0.307360,-1.956272,NaN,-1.310453
2000-08-25,1508.310059,1513.469971,1505.089966,1506.449951,685600000,7.52,1.79,8.23,2.50,6.42,...,0.000000,3.571429,NaN,-0.044307,0.000000,-0.289153,-0.129814,-2.992958,NaN,1.327854
2000-08-28,1506.449951,1523.949951,1506.449951,1514.089966,733600000,7.56,1.78,8.27,2.49,6.36,...,-2.173913,-6.896552,NaN,-0.243794,0.000000,0.114648,-0.205806,0.060496,NaN,2.558501
2000-08-29,1514.089966,1514.810059,1505.459961,1509.839966,795600000,7.58,1.77,8.30,2.49,6.30,...,0.000000,-7.407407,NaN,-0.399911,0.000000,0.154934,0.211657,2.116082,NaN,-0.456336


In [52]:
# Contar los valores nulos en cada columna
null_counts_All = AllVar_df.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts_All)

# Reemplazar los valores infinitos por NaN
AllVar_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Eliminar las filas con valores nulos o NaN
AllVar_df = AllVar_df.dropna()

# Contar de nuevo los valores nulos en cada columna
null_counts_clean = AllVar_df.isnull().sum()
print("Conteo de valores nulos por columna después de limpiar:\n", null_counts_clean)

# Guardar el DataFrame limpio a un archivo CSV
#AllVar_df.to_csv("AllVar_df_clean.csv", index=False)


Conteo de valores nulos por columna antes de limpiar:
 Open                               0
High                               0
Low                                0
Close                              0
Volume                             0
DAAA                              39
AAA10Y                            44
DBAA                              39
BAA10Y                            44
BAMLH0A0HYM2                       2
BAMLH0A2HYB                        2
BAMLH0A3HYC                        2
BAMLHE00EHYIOAS                    2
T10Y2Y                            42
T10Y3M                            42
T5YIE                            622
DEXUSEU                           53
DEXCHUS                           53
DEXCAUS                           53
DEXMXUS                           53
VIXCLS                             0
GVZCLS                          1940
Close_oil                          0
5d_moving_avg                      4
10d_moving_avg                     9
20d_moving_avg      

IMPORTANTE: cuando se unen todas las variables en AllVar_df, se cambia año de inicio al 2008 porque es cuando hay datos para todas.

In [53]:
AllVar_df.head()

,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,T10Y2Y_daily_change,T10Y3M_daily_change,T5YIE_daily_change,DEXUSEU_daily_change,DEXCHUS_daily_change,DEXCAUS_daily_change,DEXMXUS_daily_change,VIXCLS_daily_change,GVZCLS_daily_change,oil_daily_changes
Date,,,,,,,,,,,,,,,,,,,,,
2008-06-04,1376.260010,1388.180054,1371.739990,1377.199951,4338640000,5.64,1.66,7.02,3.04,6.79,...,2.721088,3.381643,-1.694915,-0.006471,0.421107,0.935230,0.291545,2.766798,-0.873744,-1.616921
2008-06-05,1377.479980,1404.050049,1377.479980,1404.050049,4350790000,5.69,1.63,7.07,3.01,6.63,...,1.986755,3.271028,2.155172,0.692422,-0.248445,0.492854,-0.077519,-10.432692,0.396651,4.488960
2008-06-06,1400.060059,1400.060059,1359.900024,1360.680054,4771660000,5.61,1.67,6.98,3.04,6.68,...,0.000000,-5.429864,3.375527,1.098972,-0.331126,-0.078470,0.543057,26.462695,3.599649,8.412233
2008-06-09,1360.829956,1370.630005,1350.619995,1361.760010,4404570000,5.58,1.56,6.96,2.94,6.58,...,-16.233766,1.913876,1.224490,-0.082639,0.000000,0.353392,-0.038580,-1.867572,3.686441,-3.024388
2008-06-10,1358.979980,1366.839966,1351.560059,1358.439941,4635070000,5.66,1.55,7.05,2.94,6.49,...,-6.976744,-1.877934,0.000000,-1.558722,0.026000,0.176074,0.231571,0.259516,3.759706,-2.262753


In [54]:
# Crear un "lag" de 1 día para los retornos diarios
AllVar_df['Lag 1_Returns'] = AllVar_df['Daily Returns'].shift(+1)
AllVar_df['Lag 2_Returns'] = AllVar_df['Daily Returns'].shift(+2)
AllVar_df['Next_Day_Returns'] = AllVar_df['Daily Returns'].shift(-1)

# Crear la columna 'TARGET' basada en 'Lagged_Daily_Returns' - Quiero predecir si el retorno del día siguiente será positivo o negativo basado con respecto al día actual en el que tomo la decision al final del día
AllVar_df['TARGET'] = np.where(AllVar_df['Next_Day_Returns'] > 0, 1, 0)

# Mostrar las primeras filas para verificar las nuevas columnas
AllVar_df.head()

,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,DEXCHUS_daily_change,DEXCAUS_daily_change,DEXMXUS_daily_change,VIXCLS_daily_change,GVZCLS_daily_change,oil_daily_changes,Lag 1_Returns,Lag 2_Returns,Next_Day_Returns,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2008-06-04,1376.260010,1388.180054,1371.739990,1377.199951,4338640000,5.64,1.66,7.02,3.04,6.79,...,0.421107,0.935230,0.291545,2.766798,-0.873744,-1.616921,NaN,NaN,1.949615,1
2008-06-05,1377.479980,1404.050049,1377.479980,1404.050049,4350790000,5.69,1.63,7.07,3.01,6.63,...,-0.248445,0.492854,-0.077519,-10.432692,0.396651,4.488960,-0.032670,NaN,-3.088921,0
2008-06-06,1400.060059,1400.060059,1359.900024,1360.680054,4771660000,5.61,1.67,6.98,3.04,6.68,...,-0.331126,-0.078470,0.543057,26.462695,3.599649,8.412233,1.949615,-0.032670,0.079369,1
2008-06-09,1360.829956,1370.630005,1350.619995,1361.760010,4404570000,5.58,1.56,6.96,2.94,6.58,...,0.000000,0.353392,-0.038580,-1.867572,3.686441,-3.024388,-3.088921,1.949615,-0.243807,0
2008-06-10,1358.979980,1366.839966,1351.560059,1358.439941,4635070000,5.66,1.55,7.05,2.94,6.49,...,0.026000,0.176074,0.231571,0.259516,3.759706,-2.262753,0.079369,-3.088921,-1.689434,0


Variables control

In [55]:
fecha_inicio = '2014-04-30'
test_size_Control=0.2

In [56]:
# Convertir a datetime
fecha_inicio_1 = pd.to_datetime(fecha_inicio)
end_date_1 = pd.to_datetime(end_date)


# Calcular el total de días
total_dias = (end_date_1 - fecha_inicio_1).days

# Calcular el número de días para el entrenamiento
dias_entrenamiento = int(total_dias * (1 - test_size_Control))

# Calcular la fecha de corte
fecha_corte = fecha_inicio_1 + pd.Timedelta(days=dias_entrenamiento)

print("Fecha de corte para dividir los datos:", fecha_corte)


Fecha de corte para dividir los datos: 2022-04-30 00:00:00


RANDOM FOREST

In [57]:
from sklearn.ensemble import RandomForestClassifier  # O RandomForestRegressor si es un problema de regresión
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [58]:
#AllVar_df =AllVar_df.loc['2019-04-03':].copy ()

#echa_inicio = '2012-01-01'
AllVar_df_Shortened = AllVar_df.loc[fecha_inicio:]

# Definir las variables independientes X y la variable dependiente Y - Uso todas las variables excepto 'Next_Day_Returns' y 'TARGET' 
#considerando que RF puede manejar mejor problem de multi colinearidad y obtener relaciones no lienales que arima, prophet o logit incidcarien mejores reusltados
X_RF = AllVar_df_Shortened.drop(columns=['Next_Day_Returns','TARGET'])
Y_RF = AllVar_df_Shortened[['TARGET']]


In [59]:
X_RF.head()

,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,T5YIE_daily_change,DEXUSEU_daily_change,DEXCHUS_daily_change,DEXCAUS_daily_change,DEXMXUS_daily_change,VIXCLS_daily_change,GVZCLS_daily_change,oil_daily_changes,Lag 1_Returns,Lag 2_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,1877.099976,1885.199951,1872.689941,1883.949951,3779230000,4.21,1.54,4.83,2.16,3.71,...,0.512821,0.449015,0.020774,-0.063851,-0.243986,-2.188184,1.371653,-1.520538,0.476076,0.323604
2014-05-01,1884.390015,1888.589966,1878.040039,1883.680054,3416740000,4.16,1.53,4.77,2.14,3.75,...,0.000000,-0.021629,0.000000,0.082147,-0.236940,-1.193139,-2.384021,-0.320834,0.299202,0.476076
2014-05-02,1885.300049,1891.329956,1878.500000,1881.140015,3159560000,4.12,1.52,4.74,2.14,3.73,...,-0.510204,-0.021634,0.000000,0.072959,-0.413714,-2.566038,-1.980198,0.341988,-0.014326,0.299202
2014-05-05,1879.449951,1885.510010,1866.770020,1884.660034,2733730000,4.16,1.53,4.78,2.15,3.74,...,-1.538462,0.144259,-0.220479,-0.164039,0.073085,2.943455,4.444444,-0.280672,-0.134845,-0.014326
2014-05-06,1883.689941,1883.689941,1867.719971,1867.719971,3327260000,4.13,1.52,4.76,2.15,3.72,...,-0.520833,0.288101,-0.317038,-0.593336,-0.042282,3.837472,-3.159252,0.020101,0.187122,-0.134845


In [60]:
Y_RF.head()

,TARGET
Date,
2014-04-30,0
2014-05-01,0
2014-05-02,1
2014-05-05,0
2014-05-06,1


In [61]:
# Asegurar que el índice está en formato datetime (si es necesario)
X_RF.index = pd.to_datetime(X_RF.index)

# Asumiendo que Y_RF está alineado con X_RF
Y_RF.index = X_RF.index  # Asegúrate de que Y_RF tenga el mismo índice que X_RF

# Dividir los datos usando la fecha de corte
train_mask = X_RF.index <= fecha_corte
test_mask = X_RF.index > fecha_corte

X_train_RF = X_RF[train_mask]
Y_train_RF = Y_RF[train_mask]
X_test_RF = X_RF[test_mask]
Y_test_RF = Y_RF[test_mask]


In [62]:
# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train_RF, X_test_RF, Y_train_RF, Y_test_RF = train_test_split(X_RF, Y_RF, test_size=test_size_Control, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train_RF)
X_train_RF_scaled = scaler.transform(X_train_RF)
X_test_RF_scaled = scaler.transform(X_test_RF)

In [63]:
X_train_RF.head()


,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,T5YIE_daily_change,DEXUSEU_daily_change,DEXCHUS_daily_change,DEXCAUS_daily_change,DEXMXUS_daily_change,VIXCLS_daily_change,GVZCLS_daily_change,oil_daily_changes,Lag 1_Returns,Lag 2_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,1877.099976,1885.199951,1872.689941,1883.949951,3779230000,4.21,1.54,4.83,2.16,3.71,...,0.512821,0.449015,0.020774,-0.063851,-0.243986,-2.188184,1.371653,-1.520538,0.476076,0.323604
2014-05-01,1884.390015,1888.589966,1878.040039,1883.680054,3416740000,4.16,1.53,4.77,2.14,3.75,...,0.000000,-0.021629,0.000000,0.082147,-0.236940,-1.193139,-2.384021,-0.320834,0.299202,0.476076
2014-05-02,1885.300049,1891.329956,1878.500000,1881.140015,3159560000,4.12,1.52,4.74,2.14,3.73,...,-0.510204,-0.021634,0.000000,0.072959,-0.413714,-2.566038,-1.980198,0.341988,-0.014326,0.299202
2014-05-05,1879.449951,1885.510010,1866.770020,1884.660034,2733730000,4.16,1.53,4.78,2.15,3.74,...,-1.538462,0.144259,-0.220479,-0.164039,0.073085,2.943455,4.444444,-0.280672,-0.134845,-0.014326
2014-05-06,1883.689941,1883.689941,1867.719971,1867.719971,3327260000,4.13,1.52,4.76,2.15,3.72,...,-0.520833,0.288101,-0.317038,-0.593336,-0.042282,3.837472,-3.159252,0.020101,0.187122,-0.134845


In [64]:
X_test_RF.head()


,Open,High,Low,Close,Volume,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,...,T5YIE_daily_change,DEXUSEU_daily_change,DEXCHUS_daily_change,DEXCAUS_daily_change,DEXMXUS_daily_change,VIXCLS_daily_change,GVZCLS_daily_change,oil_daily_changes,Lag 1_Returns,Lag 2_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-02,4130.609863,4169.810059,4062.510010,4155.379883,5163790000,4.18,1.19,5.07,2.08,4.05,...,-2.727273,-0.151846,-0.001513,0.640525,0.353652,-3.173653,1.159859,0.458492,-3.628451,2.474690
2022-05-03,4159.779785,4200.100098,4147.080078,4175.479980,4582050000,4.11,1.14,5.02,2.05,3.97,...,0.000000,0.104553,0.000000,-0.364794,-0.416034,-9.554731,-1.744766,-2.624317,0.567524,-3.628451
2022-05-04,4181.180176,4307.660156,4148.910156,4300.169922,5136360000,4.08,1.15,5.01,2.08,4.10,...,0.934579,0.066464,0.000000,-0.155800,-0.589797,-13.094017,-4.566210,5.272916,0.483713,0.567524
2022-05-05,4270.430176,4270.430176,4106.009766,4146.870117,5077030000,4.28,1.23,5.15,2.10,4.03,...,-0.308642,-0.303634,0.690083,0.249668,0.321369,22.738002,2.923977,0.417405,2.986242,0.483713
2022-05-06,4128.169922,4157.689941,4067.909912,4123.339844,5116940000,4.28,1.16,5.21,2.09,4.18,...,-0.309598,0.618635,0.174344,0.319091,-0.887093,-3.237179,-3.357438,1.394785,-3.564971,2.986242


In [65]:
Y_train_RF.head()


,TARGET
Date,
2014-04-30,0
2014-05-01,0
2014-05-02,1
2014-05-05,0
2014-05-06,1


In [66]:
Y_test_RF.head()

,TARGET
Date,
2022-05-02,1
2022-05-03,1
2022-05-04,0
2022-05-05,0
2022-05-06,0


In [67]:
# Crear una instancia del modelo Random Forest
# Ajusta los hiperparámetros como n_estimators (número de árboles) y max_depth (profundidad máxima de cada árbol) según necesites
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con el conjunto de entrenamiento escalado
rf_model.fit(X_train_RF_scaled, Y_train_RF)


c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [68]:
# Predecir las etiquetas para el conjunto de prueba
Y_pred_RF = rf_model.predict(X_test_RF_scaled)

# Calcular la precisión del modelo
accuracy = accuracy_score(Y_test_RF, Y_pred_RF)
print(f"Accuracy of the Random Forest model: {accuracy:.2f}")

# Otras métricas de evaluación
print(classification_report(Y_test_RF, Y_pred_RF))


Accuracy of the Random Forest model: 0.50
              precision    recall  f1-score   support

           0       0.50      0.86      0.63       249
           1       0.49      0.13      0.21       248

    accuracy                           0.50       497
   macro avg       0.50      0.50      0.42       497
weighted avg       0.50      0.50      0.42       497



In [69]:
Y_test_RF.head()

,TARGET
Date,
2022-05-02,1
2022-05-03,1
2022-05-04,0
2022-05-05,0
2022-05-06,0


In [70]:
#Transformo las predicciones en un DataFrame y agrupo resultados contra los valores reales
Y_pred_RF_df=pd.DataFrame(Y_pred_RF, columns=['Prediction'], index=Y_test_RF.index)
Y_pred_RF_df.head()


,Prediction
Date,
2022-05-02,0
2022-05-03,1
2022-05-04,0
2022-05-05,1
2022-05-06,1


In [71]:
result_RF = pd.merge(Y_pred_RF_df, Y_test_RF, how='left', left_index=True, right_index=True)
result_RF.head()

,Prediction,TARGET
Date,,
2022-05-02,0,1
2022-05-03,1,1
2022-05-04,0,0
2022-05-05,1,0
2022-05-06,1,0


In [72]:

# Formatear el nombre del archivo para incluir las variables 'fecha_inicio' y 'test_size_Control'
nombre_archivo_RF = f'result_RF_{fecha_inicio}_testsize{test_size_Control}.csv'

# Guardar el DataFrame en un archivo CSV con el nombre formateado
result_RF.to_csv(nombre_archivo_RF, index=True)


In [73]:
from sklearn.feature_selection import RFE


#X_train_RF_SV, X_test_RF_SV, Y_train_RF_SV, Y_test_RF_SV = train_test_split(X_RF, Y_RF, test_size=0.1, random_state=42)

X_train_RF_SV = X_RF[train_mask]
Y_train_RF_SV = Y_RF[train_mask]
X_test_RF_SV = X_RF[test_mask]
Y_test_RF_SV = Y_RF[test_mask]

# Escalar los datos
scaler = StandardScaler()
scaler.fit(X_train_RF_SV)  # Asegurarte de ajustar el scaler a X_train_RF_SV
X_train_RF_SV_scaled = scaler.transform(X_train_RF_SV)
X_test_RF_SV_scaled = scaler.transform(X_test_RF_SV)

# Verificar si X_train_RF_SV es un DataFrame y extraer los nombres de las columnas
if isinstance(X_train_RF_SV, pd.DataFrame):
    feature_names = X_train_RF_SV.columns.tolist()
else:
    raise ValueError("X_train_RF_SV debe ser un DataFrame para extraer nombres de columnas.")

# Convertir arrays de NumPy a DataFrames para facilitar la manipulación
X_train_RF_SV_scaled_df = pd.DataFrame(X_train_RF_SV_scaled, columns=feature_names)
X_test_RF_SV_scaled_df = pd.DataFrame(X_test_RF_SV_scaled, columns=feature_names)

# Crear el modelo de Random Forest
model = RandomForestClassifier(n_estimators=50, random_state=42)

# Crear el objeto RFE y clasificar las características más importantes
rfe = RFE(estimator=model, n_features_to_select=20)
rfe.fit(X_train_RF_SV_scaled_df, Y_train_RF_SV)

# Crear un DataFrame para mostrar los resultados usando los nombres de las características
results_df = pd.DataFrame({
    'Feature': feature_names,
    'Selected': rfe.support_,
    'Ranking': rfe.ranking_
})

# Filtrar para mostrar sólo las características seleccionadas
selected_features_df = results_df[results_df['Selected'] == True]

# Imprimir las características seleccionadas y su ranking
print("Selected features and their rankings:")
print(selected_features_df)


c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers

Selected features and their rankings:
                     Feature  Selected  Ranking
4                     Volume      True        1
16                   DEXUSEU      True        1
19                   DEXMXUS      True        1
21                    GVZCLS      True        1
32                    RSI_10      True        1
33               Williams_%R      True        1
36             Daily Returns      True        1
37         DAAA_daily_change      True        1
39         DBAA_daily_change      True        1
43  BAMLH0A3HYC_daily_change      True        1
46       T10Y3M_daily_change      True        1
47        T5YIE_daily_change      True        1
48      DEXUSEU_daily_change      True        1
49      DEXCHUS_daily_change      True        1
50      DEXCAUS_daily_change      True        1
51      DEXMXUS_daily_change      True        1
52       VIXCLS_daily_change      True        1
53       GVZCLS_daily_change      True        1
55             Lag 1_Returns      True        1
56

Parámetros del RandomForestClassifier
n_estimators:

Rango: (10, 1000)
Descripción: Este parámetro determina el número de árboles en el bosque. Un valor más alto generalmente mejora el rendimiento del modelo porque aumenta la diversidad y la estabilidad, pero también incrementa el tiempo de entrenamiento y la complejidad computacional.
Importancia: Aumentar el número de árboles puede reducir el riesgo de sobreajuste (overfitting), pero a costa de tiempos de cálculo mayores. La selección del valor óptimo depende del equilibrio entre rendimiento y eficiencia.
max_depth:

Rango: (1, 100)
Descripción: Define la profundidad máxima de cada árbol en el bosque. Un árbol más profundo puede capturar más detalles y complejidades en los datos, pero también puede llevar al sobreajuste.
Importancia: Controlar la profundidad de los árboles ayuda a regular el modelo. Profundidades muy grandes pueden sobreajustar el modelo a los datos de entrenamiento, mientras que profundidades muy pequeñas pueden resultar en un modelo subajustado (underfitting).
min_samples_split:

Rango: (2, 20)
Descripción: Es el número mínimo de muestras requerido para dividir un nodo interno. Si un nodo tiene menos de este número de muestras, no se dividirá.
Importancia: Este parámetro ayuda a controlar la creación de nodos adicionales en un árbol. Un valor alto evita que los árboles se vuelvan muy específicos y complejos, ayudando a prevenir el sobreajuste.
min_samples_leaf:

Rango: (1, 20)
Descripción: Es el número mínimo de muestras que debe contener un nodo hoja. Al igual que min_samples_split, un valor más alto evita la creación de nodos muy específicos.
Importancia: Establecer este valor en 1 puede permitir hojas con una sola muestra, lo que puede llevar a un ajuste excesivo. Valores más altos pueden hacer que el modelo sea más robusto y menos sensible al ruido en los datos.
max_features:

Valores: ('sqrt', 'log2', 0.1, 0.5, 0.9)
Descripción: Este parámetro controla el número máximo de características a considerar al buscar la mejor división. Puede ser un número absoluto, una fracción de características, o utilizar métodos predefinidos como sqrt (raíz cuadrada del número total de características) o log2 (logaritmo en base 2 del número total de características).
Importancia: Ajustar max_features puede ayudar a controlar el equilibrio entre la reducción del sobreajuste y la preservación de la información en los datos. Valores más bajos de max_features generalmente reducen el sobreajuste, pero pueden hacer que el modelo pase por alto patrones importantes.
Optimización Bayesiana
En este código, BayesSearchCV se utiliza para encontrar la combinación óptima de estos parámetros. BayesSearchCV es una técnica de optimización que utiliza un enfoque probabilístico para explorar el espacio de hiperparámetros, haciendo el proceso más eficiente y efectivo comparado con la búsqueda en cuadrícula (Grid Search).

Resumen
n_estimators: Controla la cantidad de árboles. Más árboles suelen mejorar el rendimiento pero aumentan el tiempo de cómputo.
max_depth: Controla la profundidad de los árboles. Profundidades mayores capturan más detalles pero pueden sobreajustar.
min_samples_split: Determina el número mínimo de muestras para dividir un nodo. Valores más altos previenen árboles demasiado específicos.
min_samples_leaf: Define el número mínimo de muestras en una hoja. Ayuda a evitar hojas con pocas muestras y reduce el sobreajuste.
max_features: Controla el número de características consideradas para cada división. Ajustar este parámetro ayuda a equilibrar la complejidad del modelo y el riesgo de sobreajuste.
A través de la optimización bayesiana, estos parámetros se ajustan de manera eficiente para mejorar el rendimiento del RandomForestClassifier de acuerdo con los datos proporcionados.

In [90]:
# Definir las variables independientes X y la variable dependiente Y

#importancia variables: 'DEXUSEU','DEXMXUS','GVZCLS','RSI_10','Williams_%R','Daily Returns','DAAA_daily_change','DBAA_daily_change','BAMLH0A3HYC_daily_change','T10Y3M_daily_change','T5YIE_daily_change','DEXUSEU_daily_change','DEXCHUS_daily_change','DEXCAUS_daily_change','DEXMXUS_daily_change','VIXCLS_daily_change','GVZCLS_daily_change','Lag 1_Returns','Lag 2_Returns' 

# Definir las columnas que queremos seleccionar
selected_columns = [
    'DEXUSEU','DEXMXUS','GVZCLS','RSI_10','Williams_%R','Daily Returns','DAAA_daily_change','DBAA_daily_change','BAMLH0A3HYC_daily_change','T10Y3M_daily_change','T5YIE_daily_change','DEXUSEU_daily_change','DEXCHUS_daily_change','DEXCAUS_daily_change','DEXMXUS_daily_change','VIXCLS_daily_change','GVZCLS_daily_change','Lag 1_Returns','Lag 2_Returns'    
]



X_RF_new = AllVar_df_Shortened[selected_columns]
Y_RF_new = AllVar_df_Shortened[['TARGET']]

In [91]:
# Asegurar que el índice está en formato datetime (si es necesario)
X_RF_new.index = pd.to_datetime(X_RF_new.index)

# Asumiendo que Y_RF_new está alineado con X_RF_new
Y_RF_new.index = X_RF_new.index  # Asegúrate de que Y_RF_new tenga el mismo índice que X_RF_new

# Dividir los datos usando la fecha de corte
train_mask = X_RF_new.index <= fecha_corte
test_mask = X_RF_new.index > fecha_corte

X_train_RF_new = X_RF_new[train_mask]
Y_train_RF_new = Y_RF_new[train_mask]
X_test_RF_new = X_RF_new[test_mask]
Y_test_RF_new = Y_RF_new[test_mask]

In [92]:
scaler = StandardScaler()
X_train_RF_new_scaled = scaler.fit_transform(X_train_RF_new)
X_test_RF_new_scaled = scaler.transform(X_test_RF_new)

In [93]:
# Crear una instancia del modelo Random Forest
# Ajusta los hiperparámetros como n_estimators (número de árboles) y max_depth (profundidad máxima de cada árbol) según necesites
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con el conjunto de entrenamiento escalado
rf_model.fit(X_train_RF_new_scaled, Y_train_RF_new)


c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [94]:
# Predecir las etiquetas para el conjunto de prueba
Y_pred_RF_new = rf_model.predict(X_test_RF_new_scaled)

# Calcular la precisión del modelo
accuracy = accuracy_score(Y_test_RF_new, Y_pred_RF_new)
print(f"Accuracy of the Random Forest model: {accuracy:.2f}")

# Otras métricas de evaluación
print(classification_report(Y_test_RF_new, Y_pred_RF_new))


Accuracy of the Random Forest model: 0.53
              precision    recall  f1-score   support

           0       0.53      0.53      0.53       249
           1       0.53      0.53      0.53       248

    accuracy                           0.53       497
   macro avg       0.53      0.53      0.53       497
weighted avg       0.53      0.53      0.53       497



In [95]:
from skopt import BayesSearchCV

# Definir la función de optimización de hiperparámetros
def optimize_rf(X_train_opt, y_train_opt):
    # Definir los parámetros para la búsqueda
    params = {
        'n_estimators': (10, 1000),
        'max_depth': (1, 100),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 20),
        'max_features': ('sqrt', 'log2', 0.1, 0.5, 0.9),
    }
    
    clf = RandomForestClassifier(random_state=42, class_weight='balanced')
    opt = BayesSearchCV(clf, params, n_iter=100, cv=5, random_state=42)
    opt.fit(X_train_opt, y_train_opt)

    best_estimator = opt.best_estimator_

    return best_estimator

# Optimización de hiperparámetros
best_estimator = optimize_rf(X_train_RF_new_scaled, Y_train_RF_new)

# Imprimir hiperparámetros del mejor estimador
print("\nHiperparámetros del mejor estimador:")
print(best_estimator.get_params())

# Entrenamiento del modelo final
best_estimator.fit(X_train_RF_new_scaled, Y_train_RF_new)

# Evaluación del modelo en el conjunto de prueba
Y_pred_RF_opt = best_estimator.predict(X_test_RF_new_scaled)
accuracy = accuracy_score(Y_test_RF_new, Y_pred_RF_opt)
print(f"\nAccuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(Y_test_RF_new, Y_pred_RF_opt))
print("Confusion Matrix:")
print(confusion_matrix(Y_test_RF_new, Y_pred_RF_opt))

c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers


Hiperparámetros del mejor estimador:
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 26, 'max_features': 0.1, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 985, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Accuracy: 0.51
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.38      0.44       249
           1       0.51      0.65      0.57       248

    accuracy                           0.51       497
   macro avg       0.51      0.51      0.50       497
weighted avg       0.51      0.51      0.50       497

Confusion Matrix:
[[ 94 155]
 [ 87 161]]


In [101]:
print(Y_pred_RF_opt)

[0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0
 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1
 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1
 1 1 1 0 1 0 0 0 0 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1
 0 1 0 0 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1
 0 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0 1 0 0 1 1 0 0 0
 0 0 1 0 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1
 1 1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1
 1 1 0 1 0 0 1 1 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 0 0 1
 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0
 0 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 0 1
 1 0 0 1 1 1 1 1 1 0 1 0 

In [102]:
Y_test_RF_new.head()

,TARGET
Date,
2022-05-02,1
2022-05-03,1
2022-05-04,0
2022-05-05,0
2022-05-06,0


In [103]:
y_pred_RF_opt_df=pd.DataFrame(Y_pred_RF_opt, columns=['Prediction'], index=Y_test_RF_new.index)   
y_pred_RF_opt_df.head()

,Prediction
Date,
2022-05-02,0
2022-05-03,1
2022-05-04,0
2022-05-05,0
2022-05-06,1


In [104]:
result_RF_opt = pd.merge(y_pred_RF_opt_df, Y_test_RF_new, how='left', left_index=True, right_index=True)
result_RF_opt.head()

,Prediction,TARGET
Date,,
2022-05-02,0,1
2022-05-03,1,1
2022-05-04,0,0
2022-05-05,0,0
2022-05-06,1,0


In [100]:

# Formatear el nombre del archivo para incluir las variables 'fecha_inicio' y 'test_size_Control'
nombre_archivo_RF_opt = f'result_RF_opt_{fecha_inicio}_testsize{test_size_Control}.csv'

# Guardar el DataFrame en un archivo CSV con el nombre formateado
result_RF_opt.to_csv(nombre_archivo_RF_opt, index=True)
